# Convolutional Layer

- toc: true
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png
- hide: true

## Introduction


## Let's Start

In [154]:
import jax 
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time 

from typing import Tuple, List, Any, Dict, Callable

### Convolutional Layer

The first function argument is the number of filters and the second argument is the shape of the kernel.  There are other parameters too, but I'm going to keep things simple for now.

In [231]:
conv_layer = tf.keras.layers.Conv2D(filters=2, kernel_size=(2, 2), use_bias=False)

Arrays fed to `layer` must be a 4D tensor where the first axis is the batch size, the second axis is the width, the third axis is the height, and thr fourth axis is 
the number of input channels.

In [232]:
input_image = np.random.randn(4,4,3)

Because Keras layers operate on batches of data, and not individual examples, a batch dimension must be addes to `input_image`.  To do this in numpy, take `input_image` and add a `None` in the axis you want to add:

In [233]:
input_batch = input_image[None,:,:,:]

In [234]:
feature_maps = conv_layer(input_batch)

In [235]:
feature_maps.shape

TensorShape([1, 3, 3, 2])

In [236]:
kernels = conv_layer.get_weights()[0]

In [237]:
print(f'w shape = {kernels.shape}')

w shape = (2, 2, 3, 2)


In [238]:
def conv2d(kernel, x):
    y = np.zeros((3,3))
    
    y[0,0] = np.sum(kernel * x[0:2,0:2])
    y[0,1] = np.sum(kernel * x[0:2,1:3])
    y[0,2] = np.sum(kernel * x[0:2,2:4])
    y[1,0] = np.sum(kernel * x[1:3,0:2])
    y[1,1] = np.sum(kernel * x[1:3,1:3])
    y[1,2] = np.sum(kernel * x[1:3,2:4])
    y[2,0] = np.sum(kernel * x[2:4,0:2])
    y[2,1] = np.sum(kernel * x[2:4,1:3])
    y[2,2] = np.sum(kernel * x[2:4,2:4])    
    
    return y

In [342]:
for i, j in zip(range(5), range(5)):
    print(i, j)

0 0
1 1
2 2
3 3
4 4


In [301]:
def conv2d_1(kernel, x):
    xm, xn = x.shape 
    km, kn = kernel.shape 
    
    y = np.zeros((xm - km + 1, xn - kn + 1))
    
    ym, yn = y.shape
    for i in range(ym):
        for j in range(yn):
            y[i, j] = np.sum(kernel * x[i:i+km, j:j+kn]) 
    
    return y

Here's a faily inefficient way to duplicate the evaluation of the `conv_layer` defined in above.  

In [343]:
def convolve(input_image, kernels):
    
    width, height, input_chans = input_image.shape 
    km, kn, _, output_chans = kernels.shape
    
    width_ = width - km + 1
    height_ = height - kn + 1 
    
    features = np.zeros((width_, height_, output_chans))

    for out_chan in range(output_chans):
        y = np.zeros((width_, height_))
        for in_chan in range(input_chans):
            y += conv2d_1(
                kernels[:,:,in_chan, out_chan], 
                input_image[:,:,in_chan]
            )
        
        # set the features
        features[:,:,out_chan] = y
    
    return features

In [340]:
features_ = convolve(input_image, kernels)[None,:,:,:]

In [341]:
assert np.all(np.isclose(feature_maps, features_))

## Max Pooling

In [272]:
pooling_layer = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(1,1))

In [273]:
yy = pooling_layer(feature_maps)
print(f'{feature_maps.shape} -> {yy.shape}')

(1, 3, 3, 2) -> (1, 2, 2, 2)


In [274]:
print(feature_maps)

tf.Tensor(
[[[[ 0.89900464 -1.0616233 ]
   [-0.10247962  1.2988867 ]
   [-0.8515937   0.6712394 ]]

  [[ 0.9654248   0.23607667]
   [-0.43404913  0.27193436]
   [-0.5712364  -0.73882663]]

  [[ 0.6847748  -0.8419535 ]
   [ 0.46285468  0.7197448 ]
   [-1.3353215   0.47066653]]]], shape=(1, 3, 3, 2), dtype=float32)


In [275]:
print(yy)

tf.Tensor(
[[[[ 0.9654248   1.2988867 ]
   [-0.10247962  1.2988867 ]]

  [[ 0.9654248   0.7197448 ]
   [ 0.46285468  0.7197448 ]]]], shape=(1, 2, 2, 2), dtype=float32)


In [279]:
print(feature_maps[0,:,:,1])

tf.Tensor(
[[-1.0616233   1.2988867   0.6712394 ]
 [ 0.23607667  0.27193436 -0.73882663]
 [-0.8419535   0.7197448   0.47066653]], shape=(3, 3), dtype=float32)


In [280]:
yy[0,:,:,1]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.2988867, 1.2988867],
       [0.7197448, 0.7197448]], dtype=float32)>

In [304]:
input_batch.shape

(1, 4, 4, 3)

In [358]:
def pool2D(x, pool_size=(2,2), strides=(1,1)):
    xm, xn = x.shape 
    pm, pn = pool_size 
    
    y = np.zeros((xm - pm + 1, xn - pn + 1))
    
    ym, yn = y.shape
    for i in range(ym):
        for j in range(yn):
            y[i, j] = np.max(x[i:i+pm, j:j+pn]) 
    
    return y

In [361]:
def pooling(features, pool_size=(2,2), strides=(1,1)):
    
    px, py = pool_size
    width, height, chans = features.shape 
    
    width_ = width - px + 1
    height_ = height - py + 1 
    
    features_ = np.zeros((width_, height_, chans))

    for chan in range(chans):
        features_[:,:,chan] = pool2D(features[:,:,chan])
    
    return features_

In [362]:
pooling(feature_maps[0,:,:,:])

array([[[ 0.96542478,  1.29888666],
        [-0.10247962,  1.29888666]],

       [[ 0.96542478,  0.7197448 ],
        [ 0.46285468,  0.7197448 ]]])

In [363]:
yy

<tf.Tensor: shape=(1, 2, 2, 2), dtype=float32, numpy=
array([[[[ 0.9654248 ,  1.2988867 ],
         [-0.10247962,  1.2988867 ]],

        [[ 0.9654248 ,  0.7197448 ],
         [ 0.46285468,  0.7197448 ]]]], dtype=float32)>